### data
TODO: choose probability/multi-hot/cnn embeddings to read<br />
TODO: choose good&bad/good&ugly to read<br />
TODO: make good&ugly data sizes same

In [1]:
import csv
import numpy as np
import pandas as pd

def read_csv(file_path, colEQ, colE, colQ):
    vecEQs = []
    vecEs = []
    vecQs = []
    labels = []
    with open(file_path, mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            
            # str -> ndarray
            vecEQ = np.array(eval(row[colEQ]))
            
            vecE_str = row[colE]
            vecE_list = vecE_str.strip('[]').split()
            vecE = np.array([float(value) for value in vecE_list])
            
            vecQ_str = row[colQ]
            vecQ_list = vecQ_str.strip('[]').split()
            vecQ = np.array([float(value) for value in vecQ_list])
            
            label = float(row['label'])
            
            # append
            vecEQs.append(vecEQ)
            vecEs.append(vecE)
            vecQs.append(vecQ)
            labels.append(label)
    return np.array(vecEQs), np.array(vecEs), np.array(vecQs), np.array(labels)

def read_pickle(file_path):
    data = pd.read_pickle(file_path)
    #display(data)
    vecEQs = np.array([np.array((x)) for x in data['predEQ']])
    vecEs = np.array([np.array((x)) for x in data['embedE']])
    vecQs = np.array([np.array((x)) for x in data['embedQ']])
    labels = data['label'].astype(float).values
    return vecEQs, vecEs, vecQs, labels

In [2]:
import random
random.seed(42)


# read prob.: good_embedding_probEQ.csv, bad_embedding_probEQ.csv

vecEQs_good, vecEs_good, vecQs_good, labels_good = read_csv('../embeddings/model_train/good_embedding_probEQ.csv', 'probEQ', 'probE', 'probQ')
#vecEQs_bad, vecEs_bad, vecQs_bad, labels_bad = read_csv('../embeddings/model_train/bad_embedding_probEQ.csv', 'probEQ', 'probE', 'probQ')
vecEQs_bad, vecEs_bad, vecQs_bad, labels_bad = read_csv('../embeddings/ugly_embedding_probEQ.csv', 'probEQ', 'probE', 'probQ')

'''
# read multi-hot: good_embedding_predEQ.csv, bad_embedding_predEQ.csv

vecEQs_good, vecEs_good, vecQs_good, labels_good = read_csv('../embeddings/model_train/good_embedding_predEQ.csv', 'predEQ', 'predE', 'predQ')
#vecEQs_bad, vecEs_bad, vecQs_bad, labels_bad = read_csv('../embeddings/model_train/bad_embedding_predEQ.csv', 'predEQ', 'predE', 'predQ')
vecEQs_bad, vecEs_bad, vecQs_bad, labels_bad = read_csv('../embeddings/ugly_embedding_predEQ.csv', 'predEQ', 'predE', 'predQ')
'''

'''
# read cnn: good_embedding.pkl, bad_embedding.pkl
vecEQs_good, vecEs_good, vecQs_good, labels_good = read_pickle('../embeddings/CNN(Resnet50)/good_embedding.pkl')
vecEQs_bad, vecEs_bad, vecQs_bad, labels_bad = read_pickle('../embeddings/CNN(Resnet50)/bad_embedding.pkl')
'''

'''
# good 和 ugly 要一樣的資料量
sampling = np.random.choice(vecEQs_good.shape[0], vecEQs_bad.shape[0], replace=False)
vecEQs_good = vecEQs_good[sampling, :]
vecEs_good = vecEs_good[sampling, :]
vecQs_good = vecQs_good[sampling, :]
labels_good = labels_good[sampling]
'''

# 合併 good & bad data, check shape
vecEQs = np.concatenate((vecEQs_good, vecEQs_bad), axis=0)
vecEs = np.concatenate((vecEs_good, vecEs_bad), axis=0)
vecQs = np.concatenate((vecQs_good, vecQs_bad), axis=0)
labels = np.concatenate((labels_good, labels_bad), axis=0)

print("vecEQs:", vecEQs.shape, "\nvecEs:", vecEs.shape, "\nvecQs:", vecQs.shape, "\nlabels:", labels.shape)

vecEQs: (3199, 196) 
vecEs: (3199, 98) 
vecQs: (3199, 98) 
labels: (3199,)


In [3]:
# X: EQ, E0, 0Q
zeros = np.zeros_like(vecEs)
x1 = vecEQs
x2 = np.hstack((vecEs, zeros))
x3 = np.hstack((zeros, vecQs))
x123 = np.vstack((x1, x2, x3))

# X^: EQ, EQ, EQ
y123 = np.vstack((x1, x1, x1))

print("X:", x123.shape, "\nX^:", y123.shape)

X: (9597, 196) 
X^: (9597, 196)


### model
TODO: 注意 training 要丟入的 x, x^

In [4]:
# 196 >> 147 >> 98 >> 147 >> 196

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

# given vecEQs
input_dim = vecEQs.shape[1]

# encoder layer (196 >> 147 >> 98)
encoding_dim1 = 147
encoding_dim2 = 98

input_layer = Input(shape=(input_dim,))
encoded1 = Dense(encoding_dim1, activation='relu')(input_layer)
encoded2 = Dense(encoding_dim2, activation='relu')(encoded1)

# decoder layer (98 >> 147 >> 196)
decoded1 = Dense(encoding_dim1, activation='relu')(encoded2)
decoded2 = Dense(input_dim, activation='sigmoid')(decoded1)

# encoder-decoder(196 >> 147 >> 98 >> 147 >> 196)
autoencoder = Model(input_layer, decoded2)

# encoder model
encoder = Model(input_layer, encoded2)

# decoder model
encoded_input = Input(shape=(encoding_dim2,))
decoder_layer1 = autoencoder.layers[-2]
decoder_layer2 = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer2(decoder_layer1(encoded_input)))

# compile model
autoencoder.compile(optimizer='adam', loss='mean_squared_error')
autoencoder.summary()

2024-08-18 01:12:05.483385: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 196)]             0         
                                                                 
 dense (Dense)               (None, 147)               28959     
                                                                 
 dense_1 (Dense)             (None, 98)                14504     
                                                                 
 dense_2 (Dense)             (None, 147)               14553     
                                                                 
 dense_3 (Dense)             (None, 196)               29008     
                                                                 
Total params: 87024 (339.94 KB)
Trainable params: 87024 (339.94 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [5]:
# training
#autoencoder.fit(vecEQs, vecEQs, epochs=50, batch_size=256, shuffle=True, validation_split=0.2)
autoencoder.fit(x123, y123, epochs=50, batch_size=256, shuffle=True, validation_split=0.2)

Epoch 1/50
30/30 [==============================] - 1s 14ms/step - loss: 0.1155 - val_loss: 0.0038
Epoch 2/50
30/30 [==============================] - 0s 5ms/step - loss: 0.0047 - val_loss: 0.0049
Epoch 3/50
30/30 [==============================] - 0s 5ms/step - loss: 0.0049 - val_loss: 0.0047
Epoch 4/50
30/30 [==============================] - 0s 4ms/step - loss: 0.0047 - val_loss: 0.0041
Epoch 5/50
30/30 [==============================] - 0s 7ms/step - loss: 0.0039 - val_loss: 0.0026
Epoch 6/50
30/30 [==============================] - 0s 4ms/step - loss: 0.0027 - val_loss: 0.0019
Epoch 7/50
30/30 [==============================] - 0s 4ms/step - loss: 0.0022 - val_loss: 0.0017
Epoch 8/50
30/30 [==============================] - 0s 4ms/step - loss: 0.0018 - val_loss: 0.0015
Epoch 9/50
30/30 [==============================] - 0s 4ms/step - loss: 0.0011 - val_loss: 0.0011
Epoch 10/50
30/30 [==============================] - 0s 4ms/step - loss: 0.0010 - val_loss: 9.9390e-04
Epoch 11/50
30

In [6]:
# middle layer
middle_vecEQs = encoder.predict(vecEQs)

100/100 [==============================] - 0s 1ms/step


### regression

In [7]:
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

X_train, X_test, y_train, y_test = train_test_split(middle_vecEQs, labels, test_size=0.2, random_state=42, stratify = labels)

# SVM model
svm_model = SVR(kernel='rbf')
svm_model.fit(X_train, y_train)

# MSE, MAE
y_pred = svm_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print(f'[SVM] MSE = {mse}, MAE = {mae}')

[SVM] MSE = 0.17691810938844715, MAE = 0.30438658214877573


In [8]:
from sklearn.ensemble import RandomForestRegressor

# Random Forest model
rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)

# MSE
y_pred = rf_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print(f'[RF] MSE = {mse}, MAE = {mae}')

[RF] MSE = 0.16146734375, MAE = 0.311328125


In [9]:
from keras.models import Sequential
from keras.layers import Dense

# Neural Network model
nn_model = Sequential()
nn_model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
nn_model.add(Dense(32, activation='relu'))
nn_model.add(Dense(16, activation='sigmoid'))
nn_model.add(Dense(1))  # output layer

nn_model.compile(optimizer='adam', loss='mean_squared_error')

# training
nn_model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=1, validation_split=0.2)

# MSE
y_pred = nn_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print(f'[NN] MSE = {mse}, MAE = {mae}')

Epoch 1/20
64/64 [==============================] - 1s 3ms/step - loss: 0.2854 - val_loss: 0.2156
Epoch 2/20
64/64 [==============================] - 0s 2ms/step - loss: 0.2111 - val_loss: 0.2089
Epoch 3/20
64/64 [==============================] - 0s 2ms/step - loss: 0.1900 - val_loss: 0.1910
Epoch 4/20
64/64 [==============================] - 0s 2ms/step - loss: 0.1737 - val_loss: 0.1853
Epoch 5/20
64/64 [==============================] - 0s 2ms/step - loss: 0.1671 - val_loss: 0.1712
Epoch 6/20
64/64 [==============================] - 0s 2ms/step - loss: 0.1669 - val_loss: 0.1710
Epoch 7/20
64/64 [==============================] - 0s 2ms/step - loss: 0.1642 - val_loss: 0.1713
Epoch 8/20
64/64 [==============================] - 0s 2ms/step - loss: 0.1643 - val_loss: 0.1717
Epoch 9/20
64/64 [==============================] - 0s 2ms/step - loss: 0.1598 - val_loss: 0.1765
Epoch 10/20
64/64 [==============================] - 0s 2ms/step - loss: 0.1615 - val_loss: 0.1688
Epoch 11/20
64/64 [

### 手動 record

In [9]:
import pandas as pd

data = {
    'probability': [0.2757061123247706, 0.2589729116945107, 0.24760135830766278],
    'multi-hot': [0.3113301803055666, 0.26886112194829226, 0.2524728533771422],
    'cnn': [0.25463821738103376, 0.23893257756563246, 0.24219229685222451]
}

df = pd.DataFrame(data, index=['svm', 'rf', 'nn'])
print("原始資料集(EQ)")
display(df)


data2 = {
    'probability': [0.2837730806873752, 0.2610291169451074, 0.2484090777202349],
    'multi-hot': [0.3188305040862045, 0.2613142604766219, 0.2565020791026134],
    'cnn': [0.23302369784904192, 0.2365852028639618, 0.24390136605141902]
}

print("\n三倍資料集(EQ, E0, 0Q)")
df2 = pd.DataFrame(data2, index=['svm', 'rf', 'nn'])
display(df2)

原始資料集(EQ)


,probability,multi-hot,cnn
svm,0.275706,0.311330,0.254638
rf,0.258973,0.268861,0.238933
nn,0.247601,0.252473,0.242192



三倍資料集(EQ, E0, 0Q)


,probability,multi-hot,cnn
svm,0.283773,0.318831,0.233024
rf,0.261029,0.261314,0.236585
nn,0.248409,0.256502,0.243901


In [45]:
# good & ugly

data3 = {
    'probability': [0.18668784933360047, 0.1716918552036199, 0.21571981347398259],
    'multi-hot': [0.16185558967853397, 0.16505469043740614, 0.1705426461395002],
    'cnn': [-1, -1, -1]
}

df3 = pd.DataFrame(data3, index=['svm', 'rf', 'nn'])
print("原始資料集(EQ)-good&ugly")
display(df3)


data4 = {
    'probability': [0.17874345017228893, 0.17046108597285067, 0.18968940847495594],
    'multi-hot': [0.17458910925342705, 0.1693084523924, 0.17449791095236505],
    'cnn': [-1, -1, -1]
}

print("\n三倍資料集(EQ, E0, 0Q)-good&ugly")
df4 = pd.DataFrame(data4, index=['svm', 'rf', 'nn'])
display(df4)

原始資料集(EQ)-good&ugly


,probability,multi-hot,cnn
svm,0.186688,0.161856,-1
rf,0.171692,0.165055,-1
nn,0.215720,0.170543,-1



三倍資料集(EQ, E0, 0Q)-good&ugly


,probability,multi-hot,cnn
svm,0.178743,0.174589,-1
rf,0.170461,0.169308,-1
nn,0.189689,0.174498,-1


### save model

In [10]:
import tensorflow as tf
import keras
import joblib

# CASE I: 訓練資料是 good=1 & bad=0
'''
# probability: 儲存原始資料集的 encoder + nn model
encoder.save('gb_prob_encoder_model.keras')
joblib.dump(nn_model, 'gb_prob_nn_model.pkl')
'''

'''
# multi-hot: 儲存原始資料集 encoder + nn model
encoder.save('gb_multihot_encoder_model.keras')
joblib.dump(nn_model, 'gb_multihot_nn_model.pkl')
'''

# CASE II: 訓練資料是 good=1 & ugly=0

# probability: 儲存三倍資料集的 autoencoder, encoder + rf model
#autoencoder.save('autoencoder_model_prob3.keras')
encoder.save('gu_prob_encoder_model.keras')
joblib.dump(rf_model, 'gu_prob_rf_model.pkl')


'''
# multi-hot: 儲存原始資料集 autoencoder, encoder + svm model
#autoencoder.save('autoencoder_model_multihot.keras')
encoder.save('gu_multihot_encoder_model.keras')
joblib.dump(svm_model, 'gu_multihot_svm_model.pkl')
'''

"\n# multi-hot: 儲存原始資料集 autoencoder, encoder + svm model\n#autoencoder.save('autoencoder_model_multihot.keras')\nencoder.save('gu_multihot_encoder_model.keras')\njoblib.dump(svm_model, 'gu_multihot_svm_model.pkl')\n"

### end